In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-text

import tensorflow_text as text
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 3.5 MB/s 
     |████████████████████████████████| 588.3 MB 19 kB/s 
     |████████████████████████████████| 6.0 MB 55.2 MB/s 
     |████████████████████████████████| 1.7 MB 70.5 MB/s 
     |████████████████████████████████| 439 kB 87.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_for_colab/tripadvisor_hotel_reviews.csv')

df = df[df.Rating != 3]

df['is_positive'] = (df['Rating'] >= 4).astype(int)

In [ ]:
df.drop(columns=['Rating'], inplace=True)

In [ ]:
df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
df

,DATA_COLUMN,LABEL_COLUMN
0,nice hotel expensive parking got good deal sta...,1
1,ok nothing special charge diamond member hilto...,0
3,"unique, great stay, wonderful time hotel monac...",1
4,"great stay great stay, went seahawk game aweso...",1
5,love monaco staff husband stayed hotel crazy w...,1
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",1
20487,great location price view hotel great quick pl...,1
20488,"ok just looks nice modern outside, desk staff ...",0
20489,hotel theft ruined vacation hotel opened sept ...,0


In [ ]:
df['LABEL_COLUMN'].value_counts()

1    15093
0     3214
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_positive = df[df['LABEL_COLUMN']==1]

In [ ]:
df_negative = df[df['LABEL_COLUMN']==0]

In [ ]:
# Тестовая выборка

In [ ]:
n_test = (df_negative.shape[0] // 4) * 3 # в оригинале (df_negative.shape[0] // 4) * 3
df_negative_test = df_negative.tail(n_test)
n_test = (df_positive.shape[0] // 20) * 3 # в оригинале df_positive.shape[0] // 20) * 3
df_positive_test = df_positive.tail(n_test)

In [ ]:
df_positive_test

,DATA_COLUMN,LABEL_COLUMN
17565,wonderful hotel wonderful town unbeknownst man...,1
17566,wonderful wonderful wonderful just returned ye...,1
17567,"affinia 50 you'all, stayed affinia 50 week, li...",1
17568,just right good location walking distance bloo...,1
17569,loved place want thank outstanding service goi...,1
...,...,...
20480,great play stay stay loyal inn package deal ha...,1
20482,great choice wife chose best western quite bit...,1
20483,good bed clean convenient just night happy sta...,1
20486,"best kept secret 3rd time staying charm, not 5...",1


In [ ]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [ ]:
df_balanced_test.sample(10)

,DATA_COLUMN,LABEL_COLUMN
17827,gaylord october 2007 overall stay gaylord suit...,1
9419,crap breakfast looked reviews couple hotels am...,0
6595,terrible place stay family miami fl plus coupl...,0
18811,"greaat time time time dominican republic, trav...",1
7814,great location air travelers planned reserved ...,0
14210,not quite par paradisus puerto rico wife child...,0
7999,horrible layne hotel lured hotel phone incredi...,0
5585,not best choice staff courteous location conve...,0
18618,okay issues husband just got 10 day stay majes...,0
10434,great location bad hotel does not live website...,0


In [ ]:
df_balanced_test['LABEL_COLUMN'].value_counts()

0    2409
1    2262
Name: LABEL_COLUMN, dtype: int64

In [ ]:
# Обучающая выборка

In [ ]:
n_train = df_negative.shape[0] // 4 # в оригианле df_negative.shape[0] // 4
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 20 # в оригинале df_positive.shape[0] // 20
df_positive_train = df_positive.head(n_train)

In [ ]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [ ]:
df_balanced_train['LABEL_COLUMN'].value_counts()

0    803
1    754
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_balanced_train.sample(10)

,DATA_COLUMN,LABEL_COLUMN
74,interesting comfortable base seattle really en...,1
843,good location adequate hotel embassy suite-bil...,1
994,just returned clift wife just returned wonderf...,1
851,nice hotel not quite ritz carlton standards rc...,1
2540,camp conquistador going caribbean presidents w...,0
2887,disgusting rooms disgusting dirty infested mos...,0
2861,kicked parents decided great sister resort fam...,0
4090,potential csme short husband went mexico city ...,0
2493,"beautiful hotel terrible houskeeping, just ret...",0
453,"old rude blah reason stayed jacuzzi suite, wis...",0


In [ ]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['LABEL_COLUMN'].squeeze()

In [ ]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['LABEL_COLUMN'].squeeze()

In [ ]:
mobilebert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3") # same as bert

In [ ]:
mobilebert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/mobilebert_en_uncased_L-24_H-128_B-512_A-4_F-4_OPT/1", trainable=True)

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = mobilebert_preprocess(text_input)
outputs = mobilebert_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)


In [ ]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
49/49 [==============================] - 924s 16s/step - loss: 530347.2500 - accuracy: 0.5183 - precision: 0.5028 - recall: 0.4682
Epoch 2/10
49/49 [==============================] - 826s 17s/step - loss: 1.2976 - accuracy: 0.5845 - precision: 0.5718 - recall: 0.5650
Epoch 3/10
49/49 [==============================] - 885s 18s/step - loss: 0.9615 - accuracy: 0.6198 - precision: 0.6098 - recall: 0.5968
Epoch 4/10
49/49 [==============================] - 864s 18s/step - loss: 0.8839 - accuracy: 0.6294 - precision: 0.6160 - recall: 0.6233
Epoch 5/10
49/49 [==============================] - 834s 17s/step - loss: 0.7507 - accuracy: 0.6602 - precision: 0.6518 - recall: 0.6406
Epoch 6/10
49/49 [==============================] - 825s 17s/step - loss: 0.6438 - accuracy: 0.7148 - precision: 0.7095 - recall: 0.6963
Epoch 7/10
49/49 [==============================] - 811s 17s/step - loss: 0.4955 - accuracy: 0.7842 - precision: 0.7757 - recall: 0.7798
Epoch 8/10
49/49 [==================

In [ ]:
y_predicted = model.predict(X_test)
y_predicted

146/146 [==============================] - 546s 4s/step


array([[0.05955774],
       [0.01809953],
       [0.00279263],
       ...,
       [0.03209257],
       [0.12130452],
       [0.99999607]], dtype=float32)

In [ ]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [ ]:
y_test

4133     0
4134     0
4135     0
4136     0
4140     0
        ..
20480    1
20482    1
20483    1
20486    1
20487    1
Name: LABEL_COLUMN, Length: 4671, dtype: int64

In [ ]:
accuracy_score(y_test, y_predicted)

0.7679297794904731

In [ ]:
precision_score(y_test, y_predicted)


0.8880105401844532

In [ ]:
recall_score(y_test, y_predicted)

0.5959328028293546

In [ ]:
f1_score(y_test, y_predicted)

0.7132275132275132

In [ ]:
df_results_for_reviews_on_hotels = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [ ]:
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'precision'] = precision_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'recall'] = recall_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'f1_score'] =  f1_score(y_test, y_predicted)


In [ ]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/mobilebert_trained_on_hotel_reviews_26_december'

In [ ]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/mobilebert_trained_on_hotel_reviews_26_december


In [ ]:
model.save(saved_model_path, include_optimizer=True) 

In [ ]:
df_results_for_reviews_on_hotels


,accuracy,precision,recall,f1_score
reviews on hotels,0.76793,0.888011,0.595933,0.713228
